In [1]:
from multiprocessing import Pool
import multiprocessing
import time
import psutil
import os
import gc
import cv2
import numpy as np
import sklearn as skl

In [2]:
gc.enable()
dir = "/home/alice/Jupyter/OpenCV/images/"
out = "/home/alice/Jupyter/OpenCV/output/"
buoy_dir = "/home/alice/Jupyter/OpenCV/Buoys/"
num_files = len(os.listdir(dir))

buoy = cv2.imread(buoy_dir+"redbuoy.png")
buoy = cv2.cvtColor(buoy, cv2.COLOR_BGR2GRAY)


In [3]:
def image_proc(queue,lock):
    time.sleep(5)
    while(len(os.listdir(out)) < num_files):
        im = None
        try:
            lock.acquire()
            im = queue.get(timeout = 5)        
            lock.release()

            if(im != None):
                index, im = im
                im = cv2.imread(im)
                im = np.array(im)[0:500][0:800]
                im = delete_blue(im)
                im = cv2.cvtColor(im, cv2.COLOR_HSV2BGR)
                im = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)

                im = cv2.GaussianBlur(im, ksize=(0,0), sigmaX=0.5, sigmaY=.75, borderType=cv2.BORDER_DEFAULT)
                im = cv2.threshold(im, 40, 255, cv2.THRESH_BINARY)[1]
                im = cv2.medianBlur(im, 3)
                im = cv2.GaussianBlur(im, ksize=(0,0), sigmaX=0.5, sigmaY=.75, borderType=cv2.BORDER_DEFAULT)
                
                err = str(im)
                im_nz = np.nonzero(im)
                err = "\n"+str(im_nz)
                im_nz = np.column_stack((im_nz[1], im_nz[0]))
                err = "\n"+str(im_nz)+"\n"
                #print(err)

                cluster(im_nz, index)

                write(im, index)
        except:
            None

    return True

#Delete all pixels that are outside of a certain red/green range
def delete_blue(image):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    image[np.logical_not(np.logical_or(np.logical_and(image[:,:,0] > 70, image[:,:,0] < 90),np.logical_or(image[:,:,0] >= 170, image[:,:,0] < 10)))] = [0,0,0]
    return(image)

def cluster(image, index):    
    if(len image > 0)
        clustering = skl.DBSCAN(eps = 1, min_samples = 3)
        clustering = clustering.fit(image)
        clustering = clustering.labels_

        clustering = np.column_stack((clustering,image))
        clustering = clustering[clustering[:,0].argsort()]
        clustering = clustering[:][clustering[:,0]!=-1]
        
        core_samples_mask = np.zeros_like(clustering)

        for i in range(0,len(unique)):
            slice = clustering[:][clustering[:,0] == i]
            slice = slice[:,1:3]
            slice = cv2.minAreaRect(slice)
            slice = cv2.boxPoints(slice)
            slice = np.int0(slice)
            print(slice)
            image = cv2.drawContours(image, [slice], 0, (0,255,255), 2)
        

#for writing one instance of an image
def write(image, index):
    cv2.imwrite(out + "out" + str(index).zfill(4) + ".jpg", image)
    return True

In [ ]:
if __name__ == '__main__':
    global num_files
    starttime = time.time()
    processes = []
    queue = multiprocessing.Queue()
    lock = multiprocessing.Lock()    
    queued = 0
    
    for i in range(0,10):
        p = multiprocessing.Process(target=image_proc, args=(queue,lock))
        processes.append(p)
        p.start()
        
    while(num_files-queued > 0):
        if(queue.qsize() < 100):
            lock.acquire()
            queue.put([str(queued),(dir+"left"+str(queued).zfill(4)+".jpg")])
            lock.release()
            queued = queued + 1
        else:
            time.sleep(1)
            
        
    for process in processes:
        process.join()
        
print('That took {} seconds'.format(time.time() - starttime))

In [ ]:
print(psutil.virtual_memory().available/1000000)